# Make intermixed (obj mask + familiar) counterbalanced sequences

These sequences are used in the final pre-registered experiment: https://osf.io/2uqz7

In [2]:
import itertools
import numpy as np
import json 
import os
import random
import copy

## Read in existing sequences

In [198]:
js_path = '/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/familiarSizeVE_MTurk/jsons'

og_sequences = {}
for seq in os.listdir(js_path):
    if 'VE250' in seq:
        file_path = js_path + '/' + seq
        # Reading the JSON data
        with open(file_path, 'r') as file:
            data = json.load(file)
            # remove catch trials from the sequence 
            fdata = [item for item in data if item['scale'] != 'NA'] 
            og_sequences[seq] = fdata


In [109]:
tseq = og_sequences['seq16_0_VE250.json']

len(tseq)

54

In [110]:
tseq_subset = tseq[0:18]
tseq_subset[0]

{'sequence': 'seq16_0_VE250',
 'image': 's27_0.75x_1.76_243.png',
 'duration': 250,
 'scene': 's27',
 'num': 0,
 'depth': 1.76,
 'scale': '0.75x',
 'ypos': 243,
 'image_path_target': 'VR_target_stimuli/s27_0.75x_1.76_243.png',
 'mask_path': 'masks/mask_0.jpg',
 'fixation_path': 'fixation.jpg'}

## Create intermixed sequences

In [199]:
def get_scale_indeces(seq_subset):
    indeces = {'0.75x': [], '1x': [], '1.3x': []}
    for index, trial in enumerate(seq_subset):
        scale = trial['scale']
        indeces[scale].append(index)

    return indeces

def update_subset(seq_subset, size):
    indeces = get_scale_indeces(seq_subset)
    for scale in indeces:
        # select three of the six elements to be made BC images
        BC_indeces = random.sample(indeces[scale], size//len(indeces)//2)
        for idx in BC_indeces:
            og_image = seq_subset[idx]['image']
            seq_subset[idx]['image'] = og_image.split('.png')[0] + '_BC.png'
            og_target_path = seq_subset[idx]['image_path_target']
            seq_subset[idx]['image_path_target'] = og_target_path.split('.png')[0] + '_BC.png'

    return seq_subset

def create_mixed_sequence(seq, num_trials, size):
    new_seq = []
    for start in range(0, num_trials, size):
        end = start + size
        subset_trials = copy.deepcopy(seq[start:end])
        new_subset = update_subset(subset_trials, size)
        new_seq += new_subset

    return new_seq

def main(og_sequences_dict, num_trials, size):
    new_sequences = {}
    for seq_name in og_sequences_dict:
        new_sequences['MX_'+seq_name] = create_mixed_sequence(og_sequences_dict[seq_name], num_trials, size)

    return new_sequences


In [200]:
num_blocks = 9
num_trials = 54
size = int(num_trials/num_blocks)

new_sequences = main(og_sequences, num_trials, size)

# x = new_tseq[48:]
# for trial in x:
#     print(trial['image'])

In [201]:
print(len(new_sequences)), print(len(set(new_sequences.keys())))

162
162


(None, None)

### Rename the sequence name within each trial

In [202]:
renamed_sequences = copy.deepcopy(new_sequences)
for seq in renamed_sequences:
    for trial in renamed_sequences[seq]:
        og_name = trial['sequence']
        trial['sequence'] = 'MX_' + og_name


## Add Catch Trials

In [203]:
catch_trial_positions = [6, 16, 25, 33, 44, 52]

In [204]:
all_catch_stim = ['c0.png', 'c1.png', 'c2.png', 'c3.png', 'c4.png', 'c5.png']

catch_trials_250 = []

for count, stim in enumerate(all_catch_stim):
    c_trial = {}
    c_trial['sequence'] = 'catch_trial'
    c_trial['image'] = stim
    
    c_trial['duration'] = 250
    
    c_trial['scene'] = 'NA'
    c_trial['num'] = count
    c_trial['depth'] = 'NA'
    c_trial['scale'] = 'NA'
    # have the image at center since there is no target
    ypos = 840//2
    c_trial['ypos'] = ypos
                
    c_trial['image_path_target'] = 'catch_stimuli/' + stim
    c_trial['mask_path'] = 'masks/' + 'mask_' + str(count) + '.jpg'
    c_trial['fixation_path'] = 'fixation.jpg'
    catch_trials_250.append(c_trial)

len(catch_trials_250), catch_trials_250[0]

(6,
 {'sequence': 'catch_trial',
  'image': 'c0.png',
  'duration': 250,
  'scene': 'NA',
  'num': 0,
  'depth': 'NA',
  'scale': 'NA',
  'ypos': 420,
  'image_path_target': 'catch_stimuli/c0.png',
  'mask_path': 'masks/mask_0.jpg',
  'fixation_path': 'fixation.jpg'})

In [205]:
sequences_with_catchTrials = {}
for seqName in renamed_sequences:
    sequence = renamed_sequences[seqName]
    for i, position in enumerate(catch_trial_positions):
        # insert catch trial into the sequence at the position 
        sequence.insert(position, catch_trials_250[i])
    sequences_with_catchTrials[seqName] = sequence

In [206]:
sequences_with_catchTrials['MX_seq0_0_VE250.json'][51:53]


[{'sequence': 'MX_seq0_0_VE250',
  'image': 's22_1x_4.29_354_BC.png',
  'duration': 250,
  'scene': 's22',
  'num': 46,
  'depth': 4.29,
  'scale': '1x',
  'ypos': 354,
  'image_path_target': 'VR_target_stimuli/s22_1x_4.29_354_BC.png',
  'mask_path': 'masks/mask_46.jpg',
  'fixation_path': 'fixation.jpg'},
 {'sequence': 'catch_trial',
  'image': 'c5.png',
  'duration': 250,
  'scene': 'NA',
  'num': 5,
  'depth': 'NA',
  'scale': 'NA',
  'ypos': 420,
  'image_path_target': 'catch_stimuli/c5.png',
  'mask_path': 'masks/mask_5.jpg',
  'fixation_path': 'fixation.jpg'}]

In [207]:
d = "s27_0.75x_1.76_243_BC.png"
if '_BC' in d:
    print(True)

d.split('_BC')

True


['s27_0.75x_1.76_243', '.png']

## Create flipped sequences: BC --> normal, normal --> BC

In [208]:

def flip_assignment(seq):
    flipped_seq = copy.deepcopy(seq)
    for trial in flipped_seq:
        og_image = trial['image']
        og_target_path = trial['image_path_target']
        if '_BC' in trial['image']:
            trial['image'] = og_image.split('_BC')[0] + '.png'
            trial['image_path_target'] = og_target_path.split('_BC')[0] + '.png'
        else:
            trial['image'] = og_image.split('.png')[0] + '_BC.png'
            trial['image_path_target'] = og_target_path.split('.png')[0] + '_BC.png'

        og_seqName = trial['sequence']
        trial['sequence'] = og_seqName + '_flipped'

    return flipped_seq

def main_flipped(og_seqs):
    flipped_sequences = {}
    for seq in og_seqs:
        f_seq = flip_assignment(og_seqs[seq])
        flipped_sequences[seq.split('.json')[0] + '_flipped.json'] = f_seq

    return flipped_sequences

In [209]:
all_flipped_seqs = main_flipped(sequences_with_catchTrials)

In [210]:
all_flipped_seqs['MX_seq0_0_VE250_flipped.json'][2]['image'], sequences_with_catchTrials['MX_seq0_0_VE250.json'][2]['image']

('s11_1.3x_5_363_BC.png', 's11_1.3x_5_363.png')

In [211]:
len(set(all_flipped_seqs.keys())), len(set(sequences_with_catchTrials.keys()))


(162, 162)

## Save sequences as jsons

In [212]:
import json
dest = '/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/MX_fS_VE_MTurk/jsons/'

for seq in all_flipped_seqs:
    with open(dest + seq, "w") as outfile:
        json.dump(all_flipped_seqs[seq], outfile)

for seq in sequences_with_catchTrials:
    with open(dest + seq, "w") as outfile:
        json.dump(sequences_with_catchTrials[seq], outfile)

# Make Counterbalancing CSV

In [191]:
import os
import csv 

In [192]:
jsons_path = '/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/MX_fS_VE_MTurk/jsons'


In [193]:
paths = []
for json in os.listdir(jsons_path):
    if '.json' in json:
        paths.append({'Path':'jsons/' + json, 'Sampled': 0})

In [194]:
# csv header
fieldnames = ['Path', 'Sampled']

# csv data
rows = paths

dest = '/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/MX_fS_VE_MTurk/'

with open(dest + 'counterbalancing.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

## Check

In [217]:
path = '/Users/prachimahableshwarkar/Documents/GW/FacialAge/FacialAge_MTurk/BNav_EC2/DepthDuration/MX_fS_VE_MTurk/jsons'

for seq in os.listdir(path):
    if '.json' in seq:
        file_path = path + '/' + seq
        # Reading the JSON data
        with open(file_path, 'r') as file:
            data = json.load(file)
            BC_count = {'0.75x': 0, '1x': 0, '1.3x': 0, 'NA': 0}
            reg_cont = {'0.75x': 0, '1x': 0, '1.3x': 0, 'NA': 0}
            for trial in data:
                if '_BC' in trial['image']:
                    BC_count[trial['scale']] += 1
                else:
                    reg_cont[trial['scale']] += 1


In [218]:
BC_count

{'0.75x': 9, '1x': 9, '1.3x': 9, 'NA': 6}

In [219]:
reg_cont

{'0.75x': 9, '1x': 9, '1.3x': 9, 'NA': 0}